### コンタンゴモデル

元記事
[A Very Different Kind of Trend Model | Following the Trend](http://www.followingthetrend.com/2016/11/a-very-different-kind-of-trend-model/)

ザックリまとめる

+ 先物とは，その商品を買っているのではなく期間を買っている．
    + 期近の金価格と，期先の金価格が違うのはそういうこと．
+ であれば，価格ではなく期間に注目するべき
+ しかも期間は常に変わるので，期間をアニュアライズして考えるべきだ．

よって
1. 期近と期先の差を取る
1. 残り期間を取得
1. 残り期間をアニュアライズ
1. 最初の取った差に↑の年率を掛ける
1. それで，３％以上であれば，Long．-3％以下でアレばShort

チェック事項
+ Threshold は 3%? (商品に寄って違うかも）
+ アニュアライズは250日？
+ 何をLong何をShort？




In [ ]:
import pandas as pd
import numpy as np
from quantopian.research.experimental import continuous_future, history

start = "2015-1-1"
end = "2017-12-12"


In [ ]:
## 取得する先物の各限月のシンボルを取得
def get_future_contract_info(root_symbol_name, year):
    month_code = list("FGHJKMNQUVXZ")
    contracts = [root_symbol_name + code + str(year) for code in month_code]
    l = list()
    for c in contracts:
        try:
            sym = symbols(c)
            l.append(sym.to_dict())
            
        except: 
            print "NoSuchSymbols", c
    df_future_contracts = pd.DataFrame(l)
    return df_future_contracts

## 各限月のプライスを取得
def get_daily_future_price(df_future_contracts, start, end):
    syms = df_future_contracts["symbol"].tolist()
    df_future_prices = history(syms, fields='price', frequency='daily', 
                     start=start, end=end)
    df_future_prices.columns = map(lambda x: x.symbol, df_future_prices.columns)
    return df_future_prices 

## 残存期間を取得して追加
def insert_remain_dates(df_future_prices, df_future_contracts):
    l = list()
    for date in df_future_prices.index:
        for expiry in df_future_contracts["expiration_date"]:
            if date < expiry:
                l.append(expiry)
                break 
                
    df_future_prices["Expiration"] = l
    df_future_prices["hoge"] = df_future_prices.index
    df_future_prices["Remain date"] = df_future_prices["Expiration"] - df_future_prices["hoge"]
    df_future_prices.drop("hoge", axis=1, inplace=True)
    return df_future_prices


## ココを直すところから
flatten = lambda *n: (e for a in n
    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))


GC15 = get_future_contract_info("GC", 15)      
GC16 = get_future_contract_info("GC", 16)      
GC17 = get_future_contract_info("GC", 17) 

df_gold_contracts = pd.concat(list((GC15, GC16, GC17)))


df_gold_future = get_daily_future_price(df_gold_contracts, start, end)
df_gold_future = insert_remain_dates(df_gold_future, df_gold_contracts)



In [ ]:

# Continuous　Futureを取得して，アニュアルDiffを出す


In [ ]:
def get_continuous_futures(root_symbol_name, start, end):
    f1 = continuous_future(root_symbol_name)
    f2 = continuous_future(root_symbol_name, offset=1)
    f3 = continuous_future(root_symbol_name, offset=2)
    continuous_futures = history([f1,f2,f3], fields='price', frequency='daily', start=start,  end=end)
    continuous_futures.columns = map(lambda x:"%s_%s" % (x.root_symbol, x.offset + 1), 
                                     continuous_futures.columns)
    return continuous_futures

gc_continuous_future = get_continuous_futures("GC", start, end)

In [ ]:
gc_continuous_future2 = pd.concat([gc_continuous_future, df_gold_future[["Expiration", "Remain date"]]], axis=1)


In [ ]:
gc_continuous_future2["diff1"] = (gc_continuous_future2["GC_2"] / gc_continuous_future2["GC_1"]).apply(np.log)
gc_continuous_future2["diff2"] = (gc_continuous_future2["GC_3"] / gc_continuous_future2["GC_2"]).apply(np.log)
gc_continuous_future2["diff1 annualize"]  = gc_continuous_future2["diff1"] * gc_continuous_future2["Remain date"].dt.days / 250
gc_continuous_future2["diff2 annualize"]  = gc_continuous_future2["diff2"] * gc_continuous_future2["Remain date"].dt.days / 250

gc_continuous_future2["diff3"] = (gc_continuous_future2["GC_3"] / gc_continuous_future2["GC_1"]).apply(np.log)
gc_continuous_future2["diff3 annualize"]  = gc_continuous_future2["diff3"] * gc_continuous_future2["Remain date"].dt.days / 250


In [ ]:
gc_continuous_future2["GC_1_pct_change"] = gc_continuous_future2["GC_1"].pct_change()
gc_continuous_future2["GC_2_pct_change"] = gc_continuous_future2["GC_2"].pct_change()
gc_continuous_future2["GC_3_pct_change"] = gc_continuous_future2["GC_3"].pct_change()


In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.cm as cm

fig, ax = plt.subplots()

im = ax.scatter(gc_continuous_future2["diff1 annualize"] - gc_continuous_future2["diff2 annualize"]
           , gc_continuous_future2["GC_1_pct_change"] - gc_continuous_future2["GC_2_pct_change"]
           , c = gc_continuous_future2["Remain date"].dt.days
           , cmap = cm.Reds
          
           )
fig.colorbar(im)
plt.xlim(-0.004, 0.006)
plt.ylim(-0.005,0.015)

#gc_continuous_future2.plot.scatter(x = "diff1 annualize", y = "GC_1_pct_change" )

In [ ]:
# gc_continuous_future2[["diff1 annualize",]].plot(secondary_y = "GC_1")
fig, ax = plt.subplots()
ax.plot(gc_continuous_future2["diff2 annualize"] - gc_continuous_future2["diff1 annualize"],)
ax2 = ax.twinx()
ax2.plot(gld.price,c = 'red')


In [ ]:
gld = get_pricing("GLD", start_date=start, end_date=end)

In [ ]:

gld.price.pct_change()[gc_continuous_future2["diff2 annualize"] - gc_continuous_future2["diff1 annualize"] < -0.003].cumsum().plot()

In [ ]:
import pandas as pd
(pd.date_range("2015-1-1","2017-12-12", ).year.strftime("")